## Load Dataset

In [1]:
from datasets import load_dataset

# Specify the dataset name and the cache directory
dataset_name = "SakanaAI/AI-CUDA-Engineer-Archive"
cache_dir = "./cache_dir"

# Load the dataset
dataset = load_dataset(dataset_name, cache_dir=cache_dir)

# Print the dataset to verify
print(dataset)

/home/abhiv/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    level_1: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12157
    })
    level_2: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', 'CUDA_Speedup_Compile', 'CUDA_Code', 'PyTorch_Code_Module', 'PyTorch_Code_Functional', 'Correct', 'Max_Diff', 'Error', 'NCU_Profile', 'Torch_Profile', 'Clang_Tidy', '__index_level_0__'],
        num_rows: 12938
    })
    level_3: Dataset({
        features: ['Op_Name', 'Level_ID', 'Task_ID', 'Kernel_Name', 'CUDA_Runtime', 'PyTorch_Native_Runtime', 'PyTorch_Compile_Runtime', 'CUDA_Speedup_Native', '

In [2]:
df_l1 = dataset["level_1"].to_pandas()

In [3]:
print(df_l1.iloc[0][["CUDA_Code"]].item())

#include <torch/extension.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <c10/cuda/CUDAException.h>

#define TILE_SIZE 16

#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_FLOAT(x) TORCH_CHECK(x.scalar_type() == torch::kFloat32, #x " must be a float32 tensor")

__global__ void matmul_tiled_kernel(const float* __restrict__ A, const float* __restrict__ B, float* __restrict__ C, int N) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float C_value = 0.0f;

    for (int m = 0; m < (N + TILE_SIZE - 1) / TILE_SIZE; ++m) {
        // Load tiles into shared memory
        if (row < N && m * TILE_SIZE + tx < N)
    

In [4]:
print(df_l1.iloc[0][["PyTorch_Code_Functional"]].item())

import torch
import torch.nn as nn
import torch.nn.functional as F


def module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """
    Performs a single square matrix multiplication (C = A * B).

    Args:
        A (torch.Tensor): Input matrix A of shape (N, N).
        B (torch.Tensor): Input matrix B of shape (N, N).

    Returns:
        torch.Tensor: Output matrix C of shape (N, N).
    """
    return torch.matmul(A, B)


class Model(nn.Module):
    """
    Simple model that performs a single square matrix multiplication (C = A * B)
    """

    def __init__(self):
        super(Model, self).__init__()

    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:
        return fn(A, B)


N = 2048


def get_inputs():
    A = torch.randn(N, N)
    B = torch.randn(N, N)
    return [A, B]


def get_init_inputs():
    return []  # No special initialization inputs needed



## Load Model

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_q14 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
model_q7 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# model_q1 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_q7,torch_dtype="auto", cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_q7, torch_dtype="auto", cache_dir=cache_dir)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.25it/s]


In [164]:
import gc
gc.collect()

import torch
with torch.cuda.device(0):  # explicitly set GPU 0 if needed
    torch.cuda.empty_cache()
    
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [5]:
import torch
device = torch.device('cuda:0')
model = model.to(device)
# tokenizer = tokenizer.to(device)

## Misc Testing

In [115]:
# Define the prompt
prompt = "what is the solution of x^2 - 2x + 1 = 0?<think>"

# prompt = "what is the second planet from the Sun?<think>"

# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the model's response
outputs = model.generate(**inputs, max_length=10_000)

# Decode the generated tokens to get the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

inputs = inputs.to('cpu')

print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

## Prompting 

In [ ]:
df_l1.iloc[0].PyTorch_Code_Functional

'import torch\nimport torch.nn as nn\nimport torch.nn.functional as F\n\n\ndef module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:\n    """\n    Performs a single square matrix multiplication (C = A * B).\n\n    Args:\n        A (torch.Tensor): Input matrix A of shape (N, N).\n        B (torch.Tensor): Input matrix B of shape (N, N).\n\n    Returns:\n        torch.Tensor: Output matrix C of shape (N, N).\n    """\n    return torch.matmul(A, B)\n\n\nclass Model(nn.Module):\n    """\n    Simple model that performs a single square matrix multiplication (C = A * B)\n    """\n\n    def __init__(self):\n        super(Model, self).__init__()\n\n    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:\n        return fn(A, B)\n\n\nN = 2048\n\n\ndef get_inputs():\n    A = torch.randn(N, N)\n    B = torch.randn(N, N)\n    return [A, B]\n\n\ndef get_init_inputs():\n    return []  # No special initialization inputs needed\n'

In [ ]:
############################################
# CUDA Prompt
############################################
# PROBLEM_STATEMENT = """You write custom CUDA kernels to replace the pytorch operators in the given architecture to get speedups. \n
#     You have complete freedom to choose the set of operators you want to replace. You may make the decision to replace some operators with custom CUDA kernels and leave others unchanged. You may replace multiple operators with custom implementations, consider operator fusion opportunities (combining multiple operators into a single kernel, for example, combining matmul+relu), or algorithmic changes (such as online softmax). You are only limited by your imagination.\n
# """
# PROBLEM_INSTRUCTION = """
# Optimize the architecture named Model with custom CUDA operators! Name your optimized output architecture ModelNew. Output the new code in codeblocks. Please generate real code, NOT pseudocode, make sure the code compiles and is fully functional. Just output the new model code, no other text, and NO testing code! \n
# """
PROBLEM_STATEMENT = """You write custom CUDA kernels to replace the pytorch operators in the given architecture to get speedups. \n
You may replace multiple operators with custom implementations, consider operator fusion opportunities (combining multiple operators into a single kernel, for example, combining matmul+relu), or algorithmic changes (such as online softmax). You are only limited by your imagination.\n
"""
PROBLEM_INSTRUCTION = """
Optimize the architecture named Model with custom CUDA operators! Name your optimized output architecture ModelNew. Output the new code in codeblocks. Please generate real code, NOT pseudocode, make sure the code compiles and is fully functional. Just output the new model code, no other text, and NO testing code! \n
Be sure to have the correct names for the cuda_source and cpp_source strings. In the ModelNew implementation, be sure to call the cuda function inside of the cuda module, not just the cuda module.
"""


def prompt_generate_custom_cuda(
    arc_src: str, example_arch_src: str, example_new_arch_src: str
) -> str:
    prompt = PROBLEM_STATEMENT

    if example_arch_src != "" and example_new_arch_src != "":
        prompt += f"""
        Here's an example to show you the syntax of the architecture you will see implemented in torch: The example architecture is for element-wise addition: \n
        ``` \n
        {example_arch_src}
        ``` \n
        The example new arch with custom CUDA kernels looks like this: 
        ```
        {example_new_arch_src}
        ``` \n
        """

    prompt += f"""
    You are given the following torch architecture: \n
    ```
    {arc_src}
    ```
    """
    prompt += PROBLEM_INSTRUCTION
    return prompt

def prompt_generate_custom_cuda_from_prompt_template(ref_arch_src: str) -> str:
    """
    Using prompt example (an element-wise addition) for prompt templates
    The most basic form of example just to show LLM the task and the expected output format
    """
    arch = ref_arch_src
    # These are strictly defined for now

    # # path to prompt template, show an example of Model (torch specifications) and ModelNew (torch + custom CUDA kernels)
    # example_arch_path = os.path.join(
    #     REPO_TOP_PATH, f"src/prompts/model_ex_add.py"
    # )
    # example_new_arch_path = os.path.join(
    #     REPO_TOP_PATH, f"src/prompts/model_new_ex_add.py"
    # )

    # if not os.path.exists(example_arch_path):
    #     raise FileNotFoundError(
    #         f"Example architecture file not found: {example_arch_path}"
    #     )
    # if not os.path.exists(example_new_arch_path):
    #     raise FileNotFoundError(
    #         f"Example new architecture file not found: {example_new_arch_path}"
    #     )

    example_arch = '''
    import torch
    import torch.nn as nn
    import torch.nn.functional as F


    class Model(nn.Module):
        def __init__(self) -> None:
            super().__init__()

        def forward(self, a, b):
            return a + b


    def get_inputs():
        # randomly generate input tensors based on the model architecture
        a = torch.randn(1, 128).cuda()
        b = torch.randn(1, 128).cuda()
        return [a, b]


    def get_init_inputs():
        # randomly generate tensors required for initialization based on the model architecture
        return []
'''
    example_new_arch = '''
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.cpp_extension import load_inline

    try:
        # Define the custom CUDA kernel for element-wise addition
        elementwise_add_source = """
        #include <torch/extension.h>
        #include <cuda_runtime.h>

        __global__ void elementwise_add_kernel(const float* a, const float* b, float* out, int size) {
            int idx = blockIdx.x * blockDim.x + threadIdx.x;
            if (idx < size) {
                out[idx] = a[idx] + b[idx];
            }
        }

        torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b) {
            auto size = a.numel();
            auto out = torch::zeros_like(a);

            const int block_size = 256;
            const int num_blocks = (size + block_size - 1) / block_size;

            elementwise_add_kernel<<<num_blocks, block_size>>>(a.data_ptr<float>(), b.data_ptr<float>(), out.data_ptr<float>(), size);

            return out;
        }
        """

        elementwise_add_cpp_source = (
            "torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b);"
        )

        # Compile the inline CUDA code for element-wise addition
        elementwise_add = load_inline(
            name="elementwise_add",
            cpp_sources=elementwise_add_cpp_source,
            cuda_sources=elementwise_add_source,
            functions=["elementwise_add_cuda"],
            verbose=True,
            extra_cflags=[""],
            extra_ldflags=[""],
        )
        
        class ModelNew(nn.Module):
        def __init__(self) -> None:
            super().__init__()
            self.elementwise_add = elementwise_add

        def forward(self, a, b):
            return self.elementwise_add.elementwise_add_cuda(a, b)
    except Exception as e:
        print("CUDA Kernel had error loading in. Probably an error in the outputted code")
        works=False

'''

    return prompt_generate_custom_cuda(arch, example_arch, example_new_arch)

In [ ]:
# Define the prompt
functional_str = df_l1.iloc[0].PyTorch_Code_Functional
prompt = prompt_generate_custom_cuda_from_prompt_template(functional_str)

# print(prompt)

# Tokenize the input promptb
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens=10_000)

# Decode the generated tokens to get the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# inputs = inputs.to('cpu')

# print(response)

In [ ]:
# print(response[response.find("</think>")+len("</think>"):])
output = response[response.find("</think>")+len("</think>"):]
output = output[output.find('import torch'):output.find('```', output.find('import torch'))]
print(output)

In [ ]:
l1_samples = df_l1[df_l1.Kernel_Name == df_l1.Op_Name]
num_correct= 0
for idx, row in l1_samples.iterrows():
    if  row.Task_ID > 2:
        break
    func_str = row.PyTorch_Code_Functional
    prompt = prompt_generate_custom_cuda_from_prompt_template(func_str)
    # Tokenize the input promptb
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    # Generate the model's response
    outputs = model.generate(**inputs, max_new_tokens=10_000)
    # Decode the generated tokens to get the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    output = response[response.find("</think>")+len("</think>"):]
    # print(func_str, output)
    print(row.Op_Name)
    output = output[output.find('import torch'):output.find('```', output.find('import torch'))]
    print(output)

    exec(func_str)
    inputs = get_inputs()
    for i in range(len(inputs)):
        inputs[i] = inputs[i].cuda()
    works=True
    try:
        exec(output)
    except Exception as e:
        print("CUDA did not compile correctly, syntax error, etc.")
        # print(e)
        works=False
    if not works:
        continue
    try:
        torchmod = Model()
        cudamod = ModelNew()
        print(torch.allclose(torchmod.forward(*inputs), cudamod.forward(*inputs), rtol=1e-1, atol=1e-3))
        if torch.allclose(torchmod.forward(*inputs), cudamod.forward(*inputs), rtol=1e-1, atol=1e-3):
            num_correct += 1
    except Exception as e:
        print("CUDA compiled but model execution had an error")
        # print(e)
    

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


1_Square_matrix_multiplication_
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline

try:
    # Define the custom CUDA kernel for matrix multiplication
    matmul_kernel_source = """
    #include <torch/extension.h>
    #include <cuda_runtime.h>

    __global__ void matmul_kernel(const float* a, const float* b, float* out, int n) {
        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;
        float sum = 0.0f;
        for (int k = 0; k < n; k++) {
            sum += a[row * n + k] * b[k * n + col];
        }
        out[row * n + col] = sum;
    }

    torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b) {
        int n = a.size(1);
        int m = b.size(0);
        int k = b.size(1);

        auto out_size = m * k;
        auto out = torch::zeros(m, k, sizeof(float));

        const int block_size = 256;
        const int num_blocks = (out_size + block_s

Using /home/abhiv/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
The input conditions for extension module matmul have changed. Bumping to version 5 and re-building as matmul_v5...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/abhiv/.cache/torch_extensions/py310_cu124/matmul/build.ninja...
/home/abhiv/miniconda3/lib/python3.10/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module matmul_v5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=matmul_v5 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/TH -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/abhiv/miniconda3/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_89,code=compute_89 -gencode=arch=compute_89,code=sm_89 --compiler-options '-fPIC' -std=c++17 -c /home/abhiv/.cache/t

In [215]:
l1_samples

,Op_Name,Level_ID,Task_ID,Kernel_Name,CUDA_Runtime,PyTorch_Native_Runtime,PyTorch_Compile_Runtime,CUDA_Speedup_Native,CUDA_Speedup_Compile,CUDA_Code,PyTorch_Code_Module,PyTorch_Code_Functional,Correct,Max_Diff,Error,NCU_Profile,Torch_Profile,Clang_Tidy,__index_level_0__
0,1_Square_matrix_multiplication_,1,1,1_Square_matrix_multiplication_,2.115,0.421087,0.445168,0.199096,0.210481,#include <torch/extension.h>\n\n#include <cuda...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport torch.nn as nn\nimport to...,True,0.001,None,None,None,None,0
99,2_Standard_matrix_multiplication_,1,2,2_Standard_matrix_multiplication_,1.942,0.425295,0.459274,0.218998,0.236495,#include <torch/extension.h>\n#include <cuda_r...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.000,None,None,None,None,0
220,3_Batched_matrix_multiplication,1,3,3_Batched_matrix_multiplication,0.808,0.128529,0.181113,0.159070,0.224150,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.000,None,None,None,None,0
341,4_Matrix_vector_multiplication_,1,4,4_Matrix_vector_multiplication_,18.158,0.061012,0.169859,0.003360,0.009354,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.012,None,None,None,None,0
462,5_Matrix_scalar_multiplication,1,5,5_Matrix_scalar_multiplication,0.229,0.181897,0.402290,0.794311,1.756726,#include <torch/extension.h>\n#include <ATen/c...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.000,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11152,96_HuberLoss,1,96,96_HuberLoss,0.012,0.016157,0.053737,1.346388,4.478060,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.000,None,None,None,None,0
11273,97_CosineSimilarityLoss,1,97,97_CosineSimilarityLoss,0.014,0.076431,0.054744,5.459388,3.910286,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.000,None,None,None,None,0
11394,98_KLDivLoss,1,98,98_KLDivLoss,0.014,0.031135,0.035177,2.223909,2.512667,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.007,None,None,None,None,0
11915,99_TripletMarginLoss,1,99,99_TripletMarginLoss,0.020,0.050352,0.034053,2.517614,1.702653,#include <torch/extension.h>\n#include <cuda.h...,import torch\nimport torch.nn as nn\n\nclass M...,import torch\nimport torch.nn as nn\nimport to...,True,0.112,None,None,None,None,0


## Sakana Eval Testing (Not Working)

In [55]:
with open(f"tasks/{df_l1.iloc[0].Op_Name}.py", "w") as f:
    f.write(df_l1.iloc[0].PyTorch_Code_Functional)
with open(f"kernels/{df_l1.iloc[0].Op_Name}.cu", "w") as f:
    f.write(df_l1.iloc[0].CUDA_Code)

File format is:

eval_kernel

task/
- torch nn module.py
- functional.py
- info.txt (dont really need this rn)

kernel/
- kernel.cu 

In [ ]:
# # Evaluation script for CUDA kernel
# # 12_Matmul_with_diagonal_matrices_
# # Evaluation script for CUDA kernel
# # 12_Matmul_with_diagonal_matrices_
# import os
# import torch
# import argparse
# from torch.utils.cpp_extension import load
# from torch.utils._pytree import tree_map
# import importlib.util
# from torch.utils.benchmark import Timer


# def easy_to_device(pytree, device):
#     return tree_map(
#         lambda x: x.to(device) if isinstance(x, torch.Tensor) else x, pytree
#     )


# def load_module_from_path(path):
#     spec = importlib.util.spec_from_file_location("module", path)
#     module = importlib.util.module_from_spec(spec)
#     spec.loader.exec_module(module)
#     return module


# def evaluate(op_name: str):
#     # parser = argparse.ArgumentParser()
#     # parser.add_argument("--op_atol", type=float, default=1e-3)
#     # parser.add_argument("--op_rtol", type=float, default=1e-1)
#     # parser.add_argument("--rep_time", type=int, default=10000)
#     # parser.add_argument("--warmup_time", type=int, default=25)
#     # args = parser.parse_args()

#     # # Get task name from info.txt
#     # with open("task/info.txt", "r") as f:
#     #     task_name = f.readline().strip()
#     #     task_name = "_".join(task_name.split("_")[1:])  # Remove problem ID

#     # Import the task module
#     # task_files = [f for f in os.listdir("tasks") if f.endswith("_functional.py")]
#     # if not task_files:
#     #     raise RuntimeError("No functional task file found")

#     task = load_module_from_path(os.path.join("tasks", op_name+'.py'))

#     # Initialize model and inputs
#     device_1 = torch.device("cuda:0")
#     torch.manual_seed(0)
#     inputs = task.get_inputs()
#     init_inputs = task.get_init_inputs()
#     model = task.Model(*init_inputs)

#     # Load CUDA kernel
#     # kernel_files = [f for f in os.listdir("kernel") if f.endswith(".cu")]
#     # if not kernel_files:
#     #     raise RuntimeError("No CUDA kernel file found")
    
#     task_name = "_".join(op_name.split("_")[1:])  # Remove problem ID
#     cuda_module = load(
#         name=task_name,
#         sources=[os.path.join("kernels", op_name+'.cu')],
#         extra_cuda_cflags=["-O3", "--use_fast_math"],
#         with_cuda=True,
#         verbose=True,
#     )

#     # Test for correctness
#     with torch.no_grad():
#         cuda_output = model.to(device_1)(
#             *easy_to_device(inputs, device_1), fn=cuda_module.forward
#         )
#         torch_output = model.to(device_1)(
#             *easy_to_device(inputs, device_1), fn=task.module_fn
#         )

#     rtol_default = 1e-1
#     atol_default = 1e-3

#     correct = torch.allclose(
#         torch_output.cpu(),
#         cuda_output.cpu(),
#         rtol=rtol_default,
#         atol=atol_default,
#     )
#     max_diff = torch.max(torch.abs(torch_output.cpu() - cuda_output.cpu())).item()
#     print(f"Tested CUDA kernel - Correct: {correct}, Max Diff: {max_diff}")

#     if correct:
#         # Evaluate CUDA kernel performance
#         cuda_timer = Timer(
#             stmt="model(*inputs, fn=cuda_module.forward)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "cuda_module": cuda_module,
#             },
#         )
#         cuda_runtime = cuda_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated CUDA kernel - Runtime: {cuda_runtime:.3f} ms")

#         # Evaluate PyTorch baseline performance
#         torch_timer = Timer(
#             stmt="model(*inputs, fn=task.module_fn)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "task": task,
#             },
#         )
#         torch_runtime = torch_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated PyTorch baseline - Runtime: {torch_runtime:.3f} ms")

#         # Evaluate torch compile performance
#         torch_fn = task.module_fn
#         compile_fn = torch.compile(torch_fn, mode="max-autotune")
#         torch_compile_timer = Timer(
#             stmt="model(*inputs, fn=compile_fn)",
#             globals={
#                 "model": model.to(device_1),
#                 "inputs": easy_to_device(inputs, device_1),
#                 "compile_fn": compile_fn,
#             },
#         )

#         torch_compile_runtime = torch_compile_timer.timeit(args.rep_time).mean * 1000
#         print(f"Evaluated torch compile - Runtime: {torch_compile_runtime:.3f} ms")

#         print(f"Speedup over PyTorch: {torch_runtime/cuda_runtime:.2f}x")
#         print(f"Speedup over torch compile: {torch_compile_runtime/cuda_runtime:.2f}x")

#         import json

#         # Store the speedup times as a json file
#         file_path = os.path.join(os.path.dirname('speedups'), f"{op_name}.json")
#         with open(file_path, "w") as f:
#             json.dump(
#                 {
#                     "max_diff": max_diff,
#                     "cuda_runtime": cuda_runtime,
#                     "torch_runtime": torch_runtime,
#                     "torch_compile_runtime": torch_compile_runtime,
#                     "speedup_over_pytorch": torch_runtime / cuda_runtime,
#                     "speedup_over_torch_compile": torch_compile_runtime / cuda_runtime,
#                 },
#                 f,
#             )
#         print(f"Speedup times stored in {file_path}")

In [114]:
# evaluate(df_l1.iloc[0].Op_Name)

## Example code from KernelBench. Evaluation seems to work with this setup.

In [110]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, a, b):
        return a + b


def get_inputs():
    # randomly generate input tensors based on the model architecture
    a = torch.randn(1, 128).cuda()
    b = torch.randn(1, 128).cuda()
    return [a, b]


def get_init_inputs():
    # randomly generate tensors required for initialization based on the model architecture
    return []


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline

# Define the custom CUDA kernel for element-wise addition
elementwise_add_source = """
#include <torch/extension.h>
#include <cuda_runtime.h>

__global__ void elementwise_add_kernel(const float* a, const float* b, float* out, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        out[idx] = a[idx] + b[idx];
    }
}

torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b) {
    auto size = a.numel();
    auto out = torch::zeros_like(a);

    const int block_size = 256;
    const int num_blocks = (size + block_size - 1) / block_size;

    elementwise_add_kernel<<<num_blocks, block_size>>>(a.data_ptr<float>(), b.data_ptr<float>(), out.data_ptr<float>(), size);

    return out;
}
"""

elementwise_add_cpp_source = (
    "torch::Tensor elementwise_add_cuda(torch::Tensor a, torch::Tensor b);"
)

# Compile the inline CUDA code for element-wise addition
elementwise_add = load_inline(
    name="elementwise_add",
    cpp_sources=elementwise_add_cpp_source,
    cuda_sources=elementwise_add_source,
    functions=["elementwise_add_cuda"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.elementwise_add = elementwise_add

    def forward(self, a, b):
        return self.elementwise_add.elementwise_add_cuda(a, b)
    
a, b = get_inputs()
torchm = Model()
cudam = ModelNew()
torch.allclose(torchm.forward(a,b), cudam.forward(a,b) )

Using /home/abhiv/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module elementwise_add, skipping build step...
Loading extension module elementwise_add...


True

In [225]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def module_fn(A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
    """
    Performs a single square matrix multiplication (C = A * B).

    Args:
        A (torch.Tensor): Input matrix A of shape (N, N).
        B (torch.Tensor): Input matrix B of shape (N, N).

    Returns:
        torch.Tensor: Output matrix C of shape (N, N).
    """
    return torch.matmul(A, B)


class Model(nn.Module):
    """
    Simple model that performs a single square matrix multiplication (C = A * B)
    """

    def __init__(self):
        super(Model, self).__init__()

    def forward(self, A: torch.Tensor, B: torch.Tensor, fn=module_fn) -> torch.Tensor:
        return fn(A, B)


N = 2048


def get_inputs():
    A = torch.randn(N, N).cuda()
    B = torch.randn(N, N).cuda()
    return [A, B]


def get_init_inputs():
    return []  # No special initialization inputs needed


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.cpp_extension import load_inline

cuda_source = """
#include <torch/extension.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <c10/cuda/CUDAException.h>

#define TILE_SIZE 16

#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)
#define CHECK_FLOAT(x) TORCH_CHECK(x.scalar_type() == torch::kFloat32, #x " must be a float32 tensor")

__global__ void matmul_tiled_kernel(const float* __restrict__ A, const float* __restrict__ B, float* __restrict__ C, int N) {
    __shared__ float As[TILE_SIZE][TILE_SIZE];
    __shared__ float Bs[TILE_SIZE][TILE_SIZE];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = blockIdx.y * TILE_SIZE + ty;
    int col = blockIdx.x * TILE_SIZE + tx;

    float C_value = 0.0f;

    for (int m = 0; m < (N + TILE_SIZE - 1) / TILE_SIZE; ++m) {
        // Load tiles into shared memory
        if (row < N && m * TILE_SIZE + tx < N)
            As[ty][tx] = A[row * N + m * TILE_SIZE + tx];
        else
            As[ty][tx] = 0.0f;

        if (col < N && m * TILE_SIZE + ty < N)
            Bs[ty][tx] = B[(m * TILE_SIZE + ty) * N + col];
        else
            Bs[ty][tx] = 0.0f;

        __syncthreads();

        // Compute partial product
        for (int k = 0; k < TILE_SIZE; ++k) {
            C_value += As[ty][k] * Bs[k][tx];
        }

        __syncthreads();
    }

    // Write the result
    if (row < N && col < N)
        C[row * N + col] = C_value;
}

torch::Tensor forward(torch::Tensor A, torch::Tensor B) {
    CHECK_INPUT(A);
    CHECK_INPUT(B);
    CHECK_FLOAT(A);
    CHECK_FLOAT(B);

    TORCH_CHECK(A.dim() == 2 && A.size(0) == A.size(1), "A must be a square matrix");
    TORCH_CHECK(B.dim() == 2 && B.size(0) == B.size(1), "B must be a square matrix");
    TORCH_CHECK(A.size(0) == B.size(0), "A and B must be of the same size");

    int64_t N = A.size(0);

    auto C = torch::zeros({N, N}, A.options());

    const float* A_data = A.data_ptr<float>();
    const float* B_data = B.data_ptr<float>();
    float* C_data = C.data_ptr<float>();

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 blocksPerGrid((N + TILE_SIZE - 1) / TILE_SIZE, (N + TILE_SIZE - 1) / TILE_SIZE);

    matmul_tiled_kernel<<<blocksPerGrid, threadsPerBlock>>>(A_data, B_data, C_data, N);

    // Check for kernel launch errors
    C10_CUDA_CHECK(cudaGetLastError());

    return C;
}
"""

cuda_cpp_source = (
    "torch::Tensor forward(torch::Tensor A, torch::Tensor B);"
)

# Compile the inline CUDA code 
cuda_mod = load_inline(
    name="matmul",
    cpp_sources=cuda_cpp_source,
    cuda_sources=cuda_source,
    functions=["forward"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cuda_mod = cuda_mod

    def forward(self, a, b):
        return self.cuda_mod.forward(a, b)

# QWEN 7B GENERATED

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.cpp_extension import load_inline

# # Define the custom CUDA kernel for matrix multiplication
# matmul_kernel_source = """
# #include <torch/extension.h>
# #include <cuda_runtime.h>

# __global__ void matmul_kernel(const float* a, const float* b, float* out, int n) {
#     int row = blockIdx.y * blockDim.y + threadIdx.y;
#     int col = blockIdx.x * blockDim.x + threadIdx.x;
#     float sum = 0.0f;
#     for (int k = 0; k < n; k++) {
#         sum += a[row * n + k] * b[k * n + col];
#     }
#     out[row * n + col] = sum;
# }

# torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b) {
#     int n = a.size(1);
#     int m = b.size(0);
#     int k = b.size(1);

#     auto out_size = m * k;
#     auto out = torch::zeros(m, k, sizeof(float));

#     const int block_size = 256;
#     const int num_blocks = (out_size + block_size - 1) / block_size;

#     matmul_kernel<<<num_blocks, block_size>>>(
#         a.data_ptr<float>(), b.data_ptr<float>(), out.data_ptr<float>(), n);

#     return out;
# }
# """

# matmul_cpp_source = (
#     "torch::Tensor matmul_cuda(torch::Tensor a, torch::Tensor b);"
# )

# # Compile the inline CUDA code for matrix multiplication
# matmul = load_inline(
#     name="matmul",
#     cpp_sources=matmul_cpp_source,
#     cuda_sources=matmul_kernel_source,
#     functions=["matmul_cuda"],
#     verbose=True,
#     extra_cflags=["-O3"],
#     extra_ldflags=[""],
# )

# class ModelNew(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         self.matmul = matmul

#     def forward(self, a, b):
#         return self.matmul.matmul_cuda(a, b)
    
torch_mod = Model()
cuda_mod = ModelNew()

a, b = get_inputs()
# a, b, = torch.eye(N).cuda(), torch.eye(N).cuda()
print(torch.allclose(torch_mod.forward(a, b), cuda_mod.forward(a, b), rtol=1e-1, atol=1e-3))
torch_mod.forward(a, b), cuda_mod.forward(a, b)

True


Using /home/abhiv/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
No modifications detected for re-loaded extension module matmul_v7, skipping build step...
Loading extension module matmul_v7...


(tensor([[-67.5708,  45.0183,  64.8336,  ..., -20.3868, -14.5004, -23.9053],
         [-56.4336, -13.6348, -25.7758,  ..., -34.6722,  29.4519,  83.3274],
         [ 86.4642, -63.3127,  -0.3111,  ...,  55.4672, -22.2791,   9.9542],
         ...,
         [-24.4993, -64.4835, -75.8926,  ...,  40.0912, -21.6326,  54.3439],
         [ 59.3703, -81.2752, -68.0154,  ...,  -2.8144, -51.7843, -17.1922],
         [100.5796,  89.1926,  56.3843,  ..., -55.6670,  40.8864,  61.0896]],
        device='cuda:0'),
 tensor([[-67.5708,  45.0183,  64.8336,  ..., -20.3868, -14.5004, -23.9053],
         [-56.4336, -13.6347, -25.7759,  ..., -34.6722,  29.4519,  83.3275],
         [ 86.4643, -63.3128,  -0.3111,  ...,  55.4672, -22.2791,   9.9542],
         ...,
         [-24.4993, -64.4835, -75.8926,  ...,  40.0912, -21.6326,  54.3439],
         [ 59.3703, -81.2752, -68.0154,  ...,  -2.8144, -51.7843, -17.1923],
         [100.5797,  89.1925,  56.3843,  ..., -55.6670,  40.8864,  61.0896]],
        device='cuda

In [231]:
exec(l1_samples.iloc[2].PyTorch_Code_Functional)
cuda_src = """
#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

// CUDA kernel for batched matrix multiplication: C = A * B
// Shapes: A (batch_size, M, K), B (batch_size, K, N), C (batch_size, M, N)
__global__ void bmm_kernel(
    const float* __restrict__ A,
    const float* __restrict__ B,
    float* __restrict__ C,
    int batch_size,
    int M,
    int K,
    int N
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = batch_size * M * N;
    if (idx >= total) return;

    int b = idx / (M * N);
    int remainder = idx % (M * N);
    int m = remainder / N;
    int n = remainder % N;

    float val = 0.0f;
    for (int k = 0; k < K; k++) {
        val += A[b * M * K + m * K + k] * B[b * K * N + k * N + n];
    }
    C[b * M * N + m * N + n] = val;
}

torch::Tensor forward_bmm(torch::Tensor A, torch::Tensor B) {
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 3, "A must be 3D");
    TORCH_CHECK(B.dim() == 3, "B must be 3D");
    TORCH_CHECK(A.size(0) == B.size(0), "Batch sizes must match");
    TORCH_CHECK(A.size(2) == B.size(1), "Inner dimensions (K) must match");

    int batch_size = A.size(0);
    int M = A.size(1);
    int K = A.size(2);
    int N = B.size(2);

    auto options = torch::TensorOptions().dtype(A.dtype()).device(A.device());
    auto C = torch::zeros({batch_size, M, N}, options);

    int total = batch_size * M * N;
    const int threads = 256;
    int blocks = (total + threads - 1) / threads;

    bmm_kernel<<<blocks, threads>>>(
        A.data_ptr<float>(),
        B.data_ptr<float>(),
        C.data_ptr<float>(),
        batch_size, M, K, N
    );

    return C;
}
"""

cuda_cpp_source = (
    "torch::Tensor forward_bmm(torch::Tensor A, torch::Tensor B);"
)

# Compile the inline CUDA code 
cuda_mod = load_inline(
    name="bmm",
    cpp_sources=cuda_cpp_source,
    cuda_sources=cuda_source,
    functions=["forward_bmm"],
    verbose=True,
    extra_cflags=[""],
    extra_ldflags=[""],
)


class ModelNew(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.cuda_mod = cuda_mod

    def forward(self, a, b):
        return self.cuda_mod.forward_bmm(a, b)

a, b = get_inputs()
a = a.cuda()
b = b.cuda()
print(torch.allclose(torch_mod.forward(a, b), cuda_mod.forward(a, b), rtol=1e-1, atol=1e-3))
torch_mod.forward(a, b), cuda_mod.forward(a, b)

Using /home/abhiv/.cache/torch_extensions/py310_cu124 as PyTorch extensions root...
The input conditions for extension module bmm have changed. Bumping to version 1 and re-building as bmm_v1...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/abhiv/.cache/torch_extensions/py310_cu124/bmm/build.ninja...
/home/abhiv/miniconda3/lib/python3.10/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module bmm_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=bmm_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/TH -isystem /home/abhiv/miniconda3/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/abhiv/miniconda3/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17  -c /home/abhiv/.cache/torch_extensions/py310_cu124/bmm/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc --generate-dependencies-with-compile --dependency-output cuda.cuda.o.d -DTORCH_EXTENSION_NAME=bmm_v1 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isy

Loading extension module bmm_v1...


ImportError: /home/abhiv/.cache/torch_extensions/py310_cu124/bmm/bmm_v1.so: undefined symbol: _Z11forward_bmmN2at6TensorES0_

## TODOs
- ~~evaluation stuff~~
- prompting qwen for good outputs 
- ~~KernelBench method - NA~~
- coding the RL portion

#include <torch/extension.h>
#include <cuda.h>
#include <cuda_runtime.h>

// CUDA kernel for batched matrix multiplication: C = A * B
// Shapes: A (batch_size, M, K), B (batch_size, K, N), C (batch_size, M, N)
__global__ void bmm_kernel(
    const float* __restrict__ A,
    const float* __restrict__ B,
    float* __restrict__ C,
    int batch_size,
    int M,
    int K,
    int N
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int total = batch_size * M * N;
    if (idx >= total) return;

    int b = idx / (M * N);
    int remainder = idx % (M * N);
    int m = remainder / N;
    int n = remainder % N;

    float val = 0.0f;
    for (int k = 0; k < K; k++) {
        val += A[b * M * K + m * K + k] * B[b * K * N + k * N + n];
    }
    C[b * M * N + m * N + n] = val;
}

torch::Tensor forward_bmm(torch::Tensor A, torch::Tensor B) {
    TORCH_CHECK(A.is_cuda(), "A must be a CUDA tensor");
    TORCH_CHECK(B.is_cuda(), "B must be a CUDA tensor");
    TORCH_CHECK(A.dim() == 3, "